In [17]:
import numpy as np
import cv2 as cv
from skimage.feature import canny

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

import os
import numpy as np
import pandas as pd
import cv2
from sklearn.preprocessing import StandardScaler
from skimage.feature import canny
import matplotlib.pyplot as plt

import joblib

## **Ekstrasi Fitur**

In [18]:
import cv2

def detect_image(image_path, size=(200, 200)):
    # Load Haar Cascade
    cascade_wajah = cv2.CascadeClassifier('assets/haarcascade_frontalface_default.xml')

    # Membaca gambar
    image = cv2.imread(image_path)  # Menggunakan variabel `image_path`
    if image is None:
        raise ValueError(f"Gambar tidak ditemukan atau tidak dapat dibaca: {image_path}")

    # Deteksi wajah
    faces = cascade_wajah.detectMultiScale(image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Wajah tidak terdeteksi
    if len(faces) == 0:
        return None
    else:
        # Cari wajah dengan frame terbesar
        largest_face = max(faces, key=lambda rect: rect[2] * rect[3])  # Pilih berdasarkan area (w * h)
        x, y, w, h = largest_face

        # Memotong gambar sesuai area deteksi wajah terbesar
        face_crop = image[y:y + h, x:x + w]
        face_crop = cv2.resize(face_crop, size)

        # Mengurangi area gambar sebesar 10% (90% dari ukuran asli)
        height, width = face_crop.shape[:2]
        new_height = int(height * 0.70)
        new_width = int(width * 0.70)

        # Menghitung margin untuk cropping agar tetap di tengah
        top_margin = (height - new_height) // 2
        left_margin = (width - new_width) // 2

        # Memotong area gambar
        face_crop = face_crop[top_margin:top_margin + new_height, left_margin:left_margin + new_width]
        return face_crop, image

In [19]:
def fetch_image(image_path, size=(200, 200)):
    # Membaca gambar
    image = cv2.imread(image_path)  # Menggunakan variabel `image_path`
    if image is None:
        raise ValueError(f"Gambar tidak ditemukan atau tidak dapat dibaca: {image_path}")
    
    # Resize ke ukuran awal
    face_crop = cv2.resize(image, size)

    # Mengurangi area gambar sebesar 30% (70% dari ukuran asli)
    height, width = face_crop.shape[:2]
    new_height = int(height * 0.70)
    new_width = int(width * 0.70)

    # Menghitung margin untuk cropping agar tetap di tengah
    top_margin = (height - new_height) // 2
    left_margin = (width - new_width) // 2

    # Memotong area gambar
    face_crop = face_crop[top_margin:top_margin + new_height, left_margin:left_margin + new_width]

    # Resize kembali hasil cropping ke ukuran `size`
    face_crop = cv2.resize(face_crop, size)

    return face_crop, image


In [33]:
def features_quadrants(img):
    """
    Calculate mean and std for 64 quadrants of the image.
    """
    h, w = img.shape  # Ambil tinggi dan lebar gambar
    h_step, w_step = h // 20, w // 20  # Ukuran setiap kuadran (dibagi 20x20)

    features = []
    for i in range(20):  # Iterasi untuk setiap baris kuadran
        for j in range(20):  # Iterasi untuk setiap kolom kuadran
            # Potong kuadran berdasarkan indeks
            quad = img[i * h_step:(i + 1) * h_step, j * w_step:(j + 1) * w_step]
            features.append(np.mean(quad))  # Hitung mean kuadran
            features.append(np.std(quad))   # Hitung std kuadran
            features.append(np.sum(quad))   # Hitung sum kuadran

    # print( "6 Features pertama: ",features[0:6])
    return np.array(features, dtype='float32')

In [21]:
def extract_feature(image_path):
    # Load gambar dari path
    img, img_org = fetch_image(image_path)
    if img is None:
        print("Image not found or cannot be loaded.")
        return None

    # Mengonversi gambar ke grayscale
    img= cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    
    # Canny edge detection
    img = canny(img, sigma=0.9)

    # Ekstraksi fitur
    img_features = features_quadrants(img)

    return img_features

In [22]:
def extract_feature_test(image_path):
    # Load gambar dari path
    img, img_org = detect_image(image_path)
    if img is None:
        print("Image not found or cannot be loaded.")
        return None

    # Mengonversi gambar ke grayscale
    img= cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    
    # Canny edge detection
    img = canny(img, sigma=0.9)

    # Ekstraksi fitur
    img_features = features_quadrants(img)

    return img_features

In [23]:
def classify_image(image_path, model, test=False):
    """
    Fungsi untuk mengklasifikasikan gambar menggunakan model yang telah dilatih.
    """
    if test==True:
        features = extract_feature_test(image_path)
    else:
        features = extract_feature(image_path)
    

    if features is None:
        return "Wajah tidak terdeteksi!"

    features = features.reshape(1, -1)
    label = model.predict(features)[0]
    return label

# Akurasi

In [24]:
import os
import pandas as pd
from sklearn.metrics import accuracy_score

# Fungsi untuk membaca dataset lokal dan menyiapkan filepath serta label
def load_dataset(folder_path):
    data = []
    for root, _, files in os.walk(folder_path):
        for file_name in files:
            if file_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                file_path = os.path.join(root, file_name)
                # Ekstrak label dari angka sebelum ekstensi
                label = int(file_name.split('_')[-1].split('.')[0])
                data.append((file_path, label))  # Path dan label
    return pd.DataFrame(data, columns=['filepath', 'label'])

# Fungsi untuk memproses dataset dan menghitung akurasi
def evaluate_model(folder_path, model):
    data = load_dataset(folder_path)
    y_true = []
    y_pred = []

    for _, row in data.iterrows():
        image_path = row['filepath']
        true_label = row['label']

        # Prediksi
        predicted_label = classify_image(image_path, model)
        
        # Menambahkan hasil prediksi ke daftar
        y_true.append(true_label)
        y_pred.append(predicted_label)

    # Hitung akurasi
    accuracy = accuracy_score(y_true, y_pred)
    return accuracy, y_true, y_pred

# Path dataset lokal
train_folder = 'dataset/final_dataset/train'
test_folder = 'dataset/final_dataset/test'

### NN

In [34]:
# Memuat model
model_path = "model/nn_20_canny_model_0.58.pkl"
model_nn = joblib.load(model_path)

# Evaluasi model dengan dataset test
accuracy, y_true, y_pred = evaluate_model(test_folder, model_nn)

# Tampilkan hasil
print(f"Akurasi Model: {accuracy * 100:.2f}%")

Akurasi Model: 34.93%


### SVM

In [36]:
# Memuat model
model_path = "model/svm_20_new_model_0.47.pkl"
model_svm = joblib.load(model_path)

# Evaluasi model dengan dataset test
accuracy, y_true, y_pred = evaluate_model(test_folder, model_svm)

# Tampilkan hasil
print(f"Akurasi Model: {accuracy * 100:.2f}%")

Akurasi Model: 24.41%


### RF

In [37]:
# Memuat model
model_path = "model/rf_20_new_model_0.55.pkl"
model_rf = joblib.load(model_path)

# Evaluasi model dengan dataset test
accuracy, y_true, y_pred = evaluate_model(test_folder, model_rf)

# Tampilkan hasil
print(f"Akurasi Model: {accuracy * 100:.2f}%")

Akurasi Model: 24.41%


# Cek

In [38]:
print("Age Range 0")
# Memanggil gambar
for i in range(20, 29):
    image_path = f'assets/test/0/{i}.jpg'  # Update this path as needed
    predicted_label_nn = classify_image(image_path, model_nn, test=True)
    predicted_label_svm = classify_image(image_path, model_svm, test=True)
    predicted_label_rf = classify_image(image_path, model_rf, test=True)
    print(f"image {i}")
    print(f"Prediksi Label NN: {predicted_label_nn}")
    print(f"Prediksi Label SVM: {predicted_label_svm}")
    print(f"Prediksi Label RF: {predicted_label_rf}")
    print()

Age Range 0
image 20
Prediksi Label NN: 5
Prediksi Label SVM: 1
Prediksi Label RF: 1

image 21
Prediksi Label NN: 1
Prediksi Label SVM: 1
Prediksi Label RF: 1

image 22
Prediksi Label NN: 5
Prediksi Label SVM: 5
Prediksi Label RF: 5

image 23
Prediksi Label NN: 1
Prediksi Label SVM: 5
Prediksi Label RF: 5

image 24
Prediksi Label NN: 5
Prediksi Label SVM: 5
Prediksi Label RF: 5

image 25
Prediksi Label NN: 5
Prediksi Label SVM: 5
Prediksi Label RF: 5

image 26
Prediksi Label NN: 1
Prediksi Label SVM: 5
Prediksi Label RF: 5

image 27
Prediksi Label NN: 1
Prediksi Label SVM: 0
Prediksi Label RF: 0

image 28
Prediksi Label NN: 5
Prediksi Label SVM: 1
Prediksi Label RF: 1



In [39]:
print("Age Range 1")
# Memanggil gambar
for i in range(20, 29):
    image_path = f'assets/test/1/{i}.jpg'  # Update this path as needed
    predicted_label_nn = classify_image(image_path, model_nn, test=True)
    predicted_label_svm = classify_image(image_path, model_svm, test=True)
    predicted_label_rf = classify_image(image_path, model_rf, test=True)
    print(f"image {i}")
    print(f"Prediksi Label NN: {predicted_label_nn}")
    print(f"Prediksi Label SVM: {predicted_label_svm}")
    print(f"Prediksi Label RF: {predicted_label_rf}")
    print()

Age Range 1
image 20
Prediksi Label NN: 1
Prediksi Label SVM: 0
Prediksi Label RF: 0

image 21
Prediksi Label NN: 1
Prediksi Label SVM: 5
Prediksi Label RF: 5

image 22
Prediksi Label NN: 5
Prediksi Label SVM: 5
Prediksi Label RF: 5

image 23
Prediksi Label NN: 1
Prediksi Label SVM: 0
Prediksi Label RF: 0

image 24
Prediksi Label NN: 1
Prediksi Label SVM: 5
Prediksi Label RF: 5

image 25
Prediksi Label NN: 5
Prediksi Label SVM: 5
Prediksi Label RF: 5

image 26
Prediksi Label NN: 1
Prediksi Label SVM: 5
Prediksi Label RF: 5

image 27
Prediksi Label NN: 5
Prediksi Label SVM: 5
Prediksi Label RF: 5

image 28
Prediksi Label NN: 5
Prediksi Label SVM: 5
Prediksi Label RF: 5



In [40]:
print("Age Range 2")
# Memanggil gambar
for i in range(20, 29):
    image_path = f'assets/test/2/{i}.jpg'  # Update this path as needed
    predicted_label_nn = classify_image(image_path, model_nn, test=True)
    predicted_label_svm = classify_image(image_path, model_svm, test=True)
    predicted_label_rf = classify_image(image_path, model_rf, test=True)
    print(f"image {i}")
    print(f"Prediksi Label NN: {predicted_label_nn}")
    print(f"Prediksi Label SVM: {predicted_label_svm}")
    print(f"Prediksi Label RF: {predicted_label_rf}")
    print()

Age Range 2
image 20
Prediksi Label NN: 5
Prediksi Label SVM: 5
Prediksi Label RF: 5

image 21
Prediksi Label NN: 1
Prediksi Label SVM: 0
Prediksi Label RF: 0

image 22
Prediksi Label NN: 5
Prediksi Label SVM: 5
Prediksi Label RF: 5

image 23
Prediksi Label NN: 1
Prediksi Label SVM: 5
Prediksi Label RF: 5

image 24
Prediksi Label NN: 5
Prediksi Label SVM: 5
Prediksi Label RF: 5

image 25
Prediksi Label NN: 1
Prediksi Label SVM: 5
Prediksi Label RF: 5

image 26
Prediksi Label NN: 5
Prediksi Label SVM: 5
Prediksi Label RF: 5

image 27
Prediksi Label NN: 1
Prediksi Label SVM: 5
Prediksi Label RF: 5

image 28
Prediksi Label NN: 5
Prediksi Label SVM: 5
Prediksi Label RF: 5



In [41]:
print("Age Range 3")
# Memanggil gambar
for i in range(20, 29):
    image_path = f'assets/test/3/{i}.jpg'  # Update this path as needed
    predicted_label_nn = classify_image(image_path, model_nn, test=True)
    predicted_label_svm = classify_image(image_path, model_svm, test=True)
    predicted_label_rf = classify_image(image_path, model_rf, test=True)
    print(f"image {i}")
    print(f"Prediksi Label NN: {predicted_label_nn}")
    print(f"Prediksi Label SVM: {predicted_label_svm}")
    print(f"Prediksi Label RF: {predicted_label_rf}")
    print()

Age Range 3
image 20
Prediksi Label NN: 5
Prediksi Label SVM: 5
Prediksi Label RF: 5

image 21
Prediksi Label NN: 5
Prediksi Label SVM: 5
Prediksi Label RF: 5

image 22
Prediksi Label NN: 5
Prediksi Label SVM: 5
Prediksi Label RF: 5

image 23
Prediksi Label NN: 1
Prediksi Label SVM: 5
Prediksi Label RF: 5

image 24
Prediksi Label NN: 0
Prediksi Label SVM: 0
Prediksi Label RF: 0

image 25
Prediksi Label NN: 5
Prediksi Label SVM: 5
Prediksi Label RF: 5

image 26
Prediksi Label NN: 5
Prediksi Label SVM: 5
Prediksi Label RF: 5

image 27
Prediksi Label NN: 5
Prediksi Label SVM: 5
Prediksi Label RF: 5

image 28
Prediksi Label NN: 0
Prediksi Label SVM: 0
Prediksi Label RF: 0



In [42]:
print("Age Range 4")
# Memanggil gambar
for i in range(20, 29):
    image_path = f'assets/test/4/{i}.jpg'  # Update this path as needed
    predicted_label_nn = classify_image(image_path, model_nn, test=True)
    predicted_label_svm = classify_image(image_path, model_svm, test=True)
    predicted_label_rf = classify_image(image_path, model_rf, test=True)
    print(f"image {i}")
    print(f"Prediksi Label NN: {predicted_label_nn}")
    print(f"Prediksi Label SVM: {predicted_label_svm}")
    print(f"Prediksi Label RF: {predicted_label_rf}")
    print()

Age Range 4
image 20
Prediksi Label NN: 0
Prediksi Label SVM: 5
Prediksi Label RF: 5

image 21
Prediksi Label NN: 5
Prediksi Label SVM: 5
Prediksi Label RF: 5

image 22
Prediksi Label NN: 4
Prediksi Label SVM: 4
Prediksi Label RF: 4

image 23
Prediksi Label NN: 1
Prediksi Label SVM: 5
Prediksi Label RF: 5

image 24
Prediksi Label NN: 5
Prediksi Label SVM: 5
Prediksi Label RF: 5

image 25
Prediksi Label NN: 5
Prediksi Label SVM: 5
Prediksi Label RF: 5

image 26
Prediksi Label NN: 5
Prediksi Label SVM: 5
Prediksi Label RF: 5

image 27
Prediksi Label NN: 5
Prediksi Label SVM: 5
Prediksi Label RF: 5

image 28
Prediksi Label NN: 1
Prediksi Label SVM: 5
Prediksi Label RF: 5



In [43]:
print("Age Range 5")
# Memanggil gambar
for i in range(20, 29):
    image_path = f'assets/test/5/{i}.jpg'  # Update this path as needed
    predicted_label_nn = classify_image(image_path, model_nn, test=True)
    predicted_label_svm = classify_image(image_path, model_svm, test=True)
    predicted_label_rf = classify_image(image_path, model_rf, test=True)
    print(f"image {i}")
    print(f"Prediksi Label NN: {predicted_label_nn}")
    print(f"Prediksi Label SVM: {predicted_label_svm}")
    print(f"Prediksi Label RF: {predicted_label_rf}")
    print()

Age Range 5
image 20
Prediksi Label NN: 5
Prediksi Label SVM: 1
Prediksi Label RF: 1

image 21
Prediksi Label NN: 1
Prediksi Label SVM: 5
Prediksi Label RF: 5

image 22
Prediksi Label NN: 1
Prediksi Label SVM: 5
Prediksi Label RF: 5

image 23
Prediksi Label NN: 5
Prediksi Label SVM: 5
Prediksi Label RF: 5

image 24
Prediksi Label NN: 5
Prediksi Label SVM: 1
Prediksi Label RF: 1

image 25
Prediksi Label NN: 0
Prediksi Label SVM: 5
Prediksi Label RF: 5

image 26
Prediksi Label NN: 3
Prediksi Label SVM: 1
Prediksi Label RF: 1

image 27
Prediksi Label NN: 5
Prediksi Label SVM: 5
Prediksi Label RF: 5

image 28
Prediksi Label NN: 5
Prediksi Label SVM: 5
Prediksi Label RF: 5

